In [ ]:
# csv 파일을 여러 번 읽어 json 파일까지 병합하는 과정

In [ ]:
# 필요한 패키지(라이브러리)를 구글 드라이버에 설치하였는가?
IS_INSTALL_ENV = True
IS_COLAB_ENV = True

In [ ]:

import os, sys  
from google.colab import drive  
drive.mount('/content/drive')   

# colab의 os 폴더 위치
pg_path = '/content/dataset' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/', pg_path)  
sys.path.insert(0, pg_path)  


# /content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/KlecSpeech/

Mounted at /content/drive


In [ ]:
# colab의 os 폴더 위치
pg_path = '/content/packages' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/packages', pg_path)  
sys.path.insert(0, pg_path)  

In [ ]:
if IS_INSTALL_ENV == False:
# !pip install transformers 대신 영구 설치
# os.symlink로 걸어놓은 위치에서 pg_path로 연결된 폴더에 영구 설치한다. 1번만 설치하면 2번 설치할 필요 없음.
    !pip install --target=$pg_path datasets
    !pip install --target=$pg_path transformers  
    !pip install --target=$pg_pat jiwer

## Create Wav2Vec2CTCTokenizer

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import tqdm
import glob

In [ ]:
dir_path = ['./dataset/KlecSpeech/csv/', './dataset/csv/']
file_prefix = ['klecspeech_ko_500', 'order_speech_ko1000']

In [ ]:
# 모든 csv 파일 리스트를 가져온다.
def get_all_filelist(dir_path):
    csv_files = []
    for i in range(len(dir_path)):
        csv_list = os.listdir(dir_path[i])
        for file in csv_list:
            csv_files.append(dir_path[i]+file)
    return csv_files



In [ ]:
# 결과를 확인한다.
all_files = get_all_filelist(dir_path)

In [ ]:
# get_filelist 
# start_idx부터 step_num 건수 만큼 파일리스트 추가
def get_filelist(csv_list, start_idx, step_num):

    end_idx = len(csv_list)
    new_list = []
    if len(csv_list) == 0:
        print("array size is 0")
        return None
    else:
        if len(csv_list) < start_idx + step_num:
            end_idx = len(csv_list)
        else:
            end_idx = start_idx + step_num

        for i in range(start_idx, end_idx):
            print("file appned : ", i, " ",csv_list[i])
            new_list.append(csv_list[i])
        return new_list

In [ ]:
def load_datasets(files):
    # files가 list로 되어 있으면 한번에 합쳐서 load해온다.
    dataset = load_dataset('csv',data_files=files, split='train', sep='\t')
    return dataset

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
import json

# 한 번 실행에 몇 개의 파일을 묶을 것인가?
num_of_files = 10

for i in range(0, len(all_files), num_of_files):
    filelist = get_filelist(all_files, i, num_of_files)
    print(filelist)
    data_csv = load_datasets(filelist)
    print(data_csv)
    remove_spectial_char_data = data_csv.map(remove_special_characters )
    df = pd.DataFrame(remove_spectial_char_data)

    char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names,
    )


    vocab_list = list(set(char_vocab["vocab"][0]))
    vocab_dict = {v: k for k, v in enumerate(vocab_list)}
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)

    print(vocab_dict)
    with open(f'vocab_{i:03d}.json_temp', 'w') as vocab_file:
        json.dump(vocab_dict, vocab_file)


file appned :  0   ./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv
file appned :  1   ./dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv
file appned :  2   ./dataset/csv/order_speech_ko1000_000.csv
file appned :  3   ./dataset/csv/order_speech_ko1000_001.csv
['./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv', './dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv', './dataset/csv/order_speech_ko1000_000.csv', './dataset/csv/order_speech_ko1000_001.csv']


Using custom data configuration default-d8ba7475a9698e83


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Parameter 'function'=<function remove_special_characters at 0x7f637043c050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d8ba7475a9698e83/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.
Dataset({
    features: ['filename', 'text', 'array'],
    num_rows: 29
})


  0%|          | 0/29 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'야': 0, '게': 1, '죠': 2, '네': 3, '에': 4, '세': 6, '무': 7, '독': 8, '오': 9, '정': 10, '침': 11, '있': 12, '런': 13, '뻐': 14, '시': 15, '랄': 16, '으': 17, '각': 18, '러': 19, '힘': 20, '팔': 21, '평': 22, '작': 23, '럽': 24, '를': 25, '안': 26, '력': 27, '선': 28, '입': 29, '궁': 30, '단': 31, '하': 32, '말': 33, '주': 34, '리': 35, '새': 36, '얀': 37, '간': 38, '잘': 39, '며': 40, '장': 41, '색': 42, '흐': 43, '못': 44, '되': 45, '알': 46, '균': 47, '로': 48, '요': 49, '동': 50, '생': 51, '줘': 52, '키': 53, '습': 54, '차': 55, '늘': 56, '절': 57, '구': 58, '을': 59, '켜': 60, '검': 61, '등': 62, '분': 63, '고': 64, '저': 65, '줄': 66, '던': 67, '그': 68, '기': 69, '내': 70, '는': 71, '졌': 72, '라': 73, '아': 74, '씀': 75, '엄': 76, '니': 77, '여': 78, '혜': 79, '막': 80, '께': 81, '않': 82, '래': 83, '점': 84, '겠': 85, '읗': 86, '지': 87, '불': 88, '까': 89, '명': 90, '마': 91, '온': 92, '싶': 93, '었': 94, '복': 95, '때': 96, '한': 97, '더': 98, '진': 99, '머': 100, '렇': 101, '우': 102, '중': 103, '금': 104, '깔': 105, '인': 106, '맙': 107, '면': 108, '봐': 109, '친': 110, '들': 11

In [ ]:
json_files = glob.glob('*.json_temp')
print(json_files)

vocab_all = {}
for filename in json_files:
    with open(filename, 'r') as vocab_file:
        vocab_dict = json.load(vocab_file)
        print(type(vocab_dict))
        vocab_all.update(vocab_dict)

# 번호 재배열
vocab_all = {v: k for k, v in enumerate(vocab_all)}
print(len(vocab_all))

# print(vocab_all)

with open(f'vocab.json', 'w') as vocab_file:  
    json.dump(vocab_all, vocab_file)


['vocab_000.json_temp']
<class 'dict'>
195
{'야': 0, '게': 1, '죠': 2, '네': 3, '에': 4, '세': 5, '무': 6, '독': 7, '오': 8, '정': 9, '침': 10, '있': 11, '런': 12, '뻐': 13, '시': 14, '랄': 15, '으': 16, '각': 17, '러': 18, '힘': 19, '팔': 20, '평': 21, '작': 22, '럽': 23, '를': 24, '안': 25, '력': 26, '선': 27, '입': 28, '궁': 29, '단': 30, '하': 31, '말': 32, '주': 33, '리': 34, '새': 35, '얀': 36, '간': 37, '잘': 38, '며': 39, '장': 40, '색': 41, '흐': 42, '못': 43, '되': 44, '알': 45, '균': 46, '로': 47, '요': 48, '동': 49, '생': 50, '줘': 51, '키': 52, '습': 53, '차': 54, '늘': 55, '절': 56, '구': 57, '을': 58, '켜': 59, '검': 60, '등': 61, '분': 62, '고': 63, '저': 64, '줄': 65, '던': 66, '그': 67, '기': 68, '내': 69, '는': 70, '졌': 71, '라': 72, '아': 73, '씀': 74, '엄': 75, '니': 76, '여': 77, '혜': 78, '막': 79, '께': 80, '않': 81, '래': 82, '점': 83, '겠': 84, '읗': 85, '지': 86, '불': 87, '까': 88, '명': 89, '마': 90, '온': 91, '싶': 92, '었': 93, '복': 94, '때': 95, '한': 96, '더': 97, '진': 98, '머': 99, '렇': 100, '우': 101, '중': 102, '금': 103, '깔': 104, '인': 105, '맙': 1

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

In [ ]:
# 여기까지 되는 것은 확인, 나머지는 테스트 중
print(tokenizer)

PreTrainedTokenizer(name_or_path='', vocab_size=195, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})
